## Quick Tour

The following examples show how to get started with the `unstructured` library. See
our [documentation page](https://unstructured-io.github.io/unstructured) for a full description
of the features in the library.

Another way to try out the `unstructured` library is by running a docker container -- compatible with either Intel/AMD or Apple Silicon! Check out the [instructions for using the docker image](https://github.com/Unstructured-IO/unstructured#dizzy-instructions-for-using-the-docker-image).

In [ ]:
#%reset
from google.colab import drive
drive.mount('/content/gdrive')
import requests
import json
import numpy as np
import pandas as pd
from datetime import datetime as dt

Mounted at /content/gdrive


In [ ]:
# Install Requirements
!apt-get -qq install poppler-utils tesseract-ocr
# Upgrade Pillow to latest version
#%pip install -q --user --upgrade pillow
%pip install -q pillow==10.0.1
# Install Python Packages
%pip install -q unstructured["all-docs"]==0.12.5
# NOTE: you may also upgrade to the latest version with the command below,
#       though a more recent version of unstructured will not have been tested with this notebook
# %pip install -q --upgrade unstructured

Selecting previously unselected package poppler-utils.
(Reading database ... 121920 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.3_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.3) ...
Selecting previously unselected package tesseract-ocr-eng.
Preparing to unpack .../tesseract-ocr-eng_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-eng (1:4.00~git30-7274cfa-1.1) ...
Selecting previously unselected package tesseract-ocr-osd.
Preparing to unpack .../tesseract-ocr-osd_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-osd (1:4.00~git30-7274cfa-1.1) ...
Selecting previously unselected package tesseract-ocr.
Preparing to unpack .../tesseract-ocr_4.1.1-2.1build1_amd64.deb ...
Unpacking tesseract-ocr (4.1.1-2.1build1) ...
Setting up tesseract-ocr-eng (1:4.00~git30-7274cfa-1.1) ...
Setting up tesseract-ocr-osd (1:4.00~git30-7274cfa-1.1) ...
Setting up poppler-utils (22.02.0-2ubuntu0.3) ...
Setting up tess

See our [example docs page](https://github.com/Unstructured-IO/unstructured/tree/main/example-docs) to find example docs used in this tutorial. You can also upload your own files by clicking on “Choose Files” on the left panel then select and upload the file to Colab.

In [ ]:
!mkdir -p example-docs
# Install example-10k.html and layout-parser-paper.pdf
!wget  https://raw.githubusercontent.com/Unstructured-IO/unstructured/main/example-docs/example-10k.html -P example-docs
!wget  https://raw.githubusercontent.com/Unstructured-IO/unstructured/main/example-docs/layout-parser-paper-fast.pdf -P example-docs

--2024-05-03 11:41:02--  https://raw.githubusercontent.com/Unstructured-IO/unstructured/main/example-docs/example-10k.html
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2456707 (2.3M) [text/plain]
Saving to: ‘example-docs/example-10k.html’

example-10k.html    100%[===================>]   2.34M  --.-KB/s    in 0.04s   

2024-05-03 11:41:03 (52.5 MB/s) - ‘example-docs/example-10k.html’ saved [2456707/2456707]

--2024-05-03 11:41:03--  https://raw.githubusercontent.com/Unstructured-IO/unstructured/main/example-docs/layout-parser-paper-fast.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... conn

In [ ]:
# Install NLTK Data
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

### HTML Parsing

You can parse an HTML document using the following workflow:

In [ ]:
from unstructured.documents.html import HTMLDocument

doc = HTMLDocument.from_file("example-docs/example-10k.html")

# This is how you would use a document from your google Drive
"""
from google.colab import drive
drive.mount('/content/drive/')
doc = HTMLDocument.from_file("drive/MyDrive/your-filename.html")
"""

'\nfrom google.colab import drive\ndrive.mount(\'/content/drive/\')\ndoc = HTMLDocument.from_file("drive/MyDrive/your-filename.html")\n'

The third page of output looks like the following:

In [ ]:
print(doc.pages[2])

SPECIAL NOTE REGARDING FORWARD-LOOKING STATEMENTS

This report contains statements that do not relate to historical or current facts but are “forward-looking” statements. These statements relate to analyses and other information based on forecasts of future results and estimates of amounts not yet determinable. These statements may also relate to future events or trends, our future prospects and proposed new products, services, developments or business strategies, among other things. These statements can generally (although not always) be identified by their use of terms and phrases such as anticipate, appear, believe, could, would, estimate, expect, indicate, intent, may, plan, predict, project, pursue, will continue and other similar terms and phrases, as well as the use of the future tense.

Actual results could differ materially from those expressed or implied in our forward-looking statements. Our future financial condition and results of operations, as well as any forward-looking

In [ ]:
doc.pages[2].elements

You can see that the parser successfully differentiated between titles and narrative text.

### PDF Parsing

There are two strategies availalbe for parsing PDF documents: "fast" and "hi_res." The default strategy is "hi_res"

If your main objective is extracting text from a "clean" PDF, i.e. one that does not include text in images that require OCR), go with the "fast" option.

Otherwise, if your PDF may have images with text to extract, or, you prefer to have better structured Elements that better characterize the text items within the document, go with with the "hi_res" option.

Naturally, "fast" is faster than "hi_res" -- by an order of magnitude!

In [ ]:
from unstructured.partition.pdf import partition_pdf

elements = partition_pdf("gdrive/MyDrive/Colab Notebooks/Unstructured_io/India_Art_IV_2023.pdf")

elements_fast = partition_pdf("gdrive/MyDrive/Colab Notebooks/Unstructured_io/India_Art_IV_2023.pdf", strategy="fast")

Let's examine the types of elements returned for both the "hi_res" and "fast" strategies:

In [ ]:
from collections import Counter

display(Counter(type(element) for element in elements))
print("")
# The composition of elements can be different for elements derived with the "fast" strategy
display(Counter(type(element) for element in elements_fast))

Counter({unstructured.documents.elements.Title: 1055,
         unstructured.documents.elements.Text: 2161,
         unstructured.documents.elements.NarrativeText: 841,
         unstructured.documents.elements.ListItem: 687,
         unstructured.documents.elements.Footer: 161,
         unstructured.documents.elements.Header: 127})

Counter({unstructured.documents.elements.Title: 1055,
         unstructured.documents.elements.Text: 2161,
         unstructured.documents.elements.NarrativeText: 841,
         unstructured.documents.elements.ListItem: 687,
         unstructured.documents.elements.Footer: 161,
         unstructured.documents.elements.Header: 127})

Let's display the type and text of some of the elements in the document:

In [ ]:
display(*[(type(element), element.text) for element in elements[10:13]])

(unstructured.documents.elements.NarrativeText,
 'Copies of this report are available to the public from')

(unstructured.documents.elements.Text,
 'International Monetary Fund • Publication Services PO Box 92780 • Washington, D.C. 20090 Telephone: (202) 623-7430 • Fax: (202) 623-7201 E-mail: publications@imf.org Web: http://www.imf.org')

(unstructured.documents.elements.Title,
 'International Monetary Fund Washington, D.C.')

You can see that the parser also successfully differentiated between titles and narrative text from a PDF file. However, be aware that element classification is improving as the library evolves, tends to be more accurate with the "hi_res" strategy, and may not always correct.

Now we can join the elements and print the extracted texts from the PDF

In [ ]:
print("\n\n".join([str(el) for el in elements]))

December 2023

IMF Country Report No. 23/426

INDIA 2023 ARTICLE IV CONSULTATION—PRESS RELEASE; STAFF REPORT; AND STATEMENT BY THE EXECUTIVE DIRECTOR FOR INDIA

Under Article IV of the IMF’s Articles of Agreement, the IMF holds bilateral discussions with members, usually every year. In the context of the 2023 Article IV consultation with India, the following documents have been released and are included in this package:

A Press Release summarizing the views of the Executive Board as expressed during its November 20, 2023 consideration of the staff report that concluded the Article IV consultation with India.

The Staff Report prepared by a staff team of the IMF for the Executive Board’s

consideration on November 20, 2023, following discussions that ended on September 21, 2023, with the officials of India on economic developments and policies. Based on information available at the time of these discussions, the staff report was completed on November 3, 2023.

An Informational Annex pr

In [ ]:
for i in range(len(elements[:100])):
  print('#########################')
  print(i)
  print(elements[i].text)

#########################
0
December 2023
#########################
1
IMF Country Report No. 23/426
#########################
2
INDIA 2023 ARTICLE IV CONSULTATION—PRESS RELEASE; STAFF REPORT; AND STATEMENT BY THE EXECUTIVE DIRECTOR FOR INDIA
#########################
3
Under Article IV of the IMF’s Articles of Agreement, the IMF holds bilateral discussions with members, usually every year. In the context of the 2023 Article IV consultation with India, the following documents have been released and are included in this package:
#########################
4
A Press Release summarizing the views of the Executive Board as expressed during its November 20, 2023 consideration of the staff report that concluded the Article IV consultation with India.
#########################
5
The Staff Report prepared by a staff team of the IMF for the Executive Board’s
#########################
6
consideration on November 20, 2023, following discussions that ended on September 21, 2023, with the officials o

In [ ]:
tables = [el for el in elements if el.category == "Table"]
tables

[]

### PDF Parcing Reading tables in pdf with only partition(not partition_pdf) with startegy = 'hi_res'

In [ ]:
from unstructured.partition.auto import partition

filename = 'gdrive/MyDrive/Colab Notebooks/Unstructured_io/India_Art_IV_2023.pdf'

elements = partition(filename=filename,
                     strategy='hi_res',
           )

tables = [el for el in elements if el.category == "Table"]

print(tables[0].text)
print(tables[0].metadata.text_as_html)

yolox_l0.05.onnx:   0%|          | 0.00/217M [00:00<?, ?B/s]

In [ ]:
tables[6].text

In [ ]:
tables[6].metadata.text_as_html

In [ ]:
tables[7].text